In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from zipfile import ZipFile
from io import BytesIO
from urllib.request import urlopen
from collections import Counter
from sklearn.metrics import confusion_matrix

from statsmodels.formula.api import logit
from statsmodels.api import Logit

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeRegressor

from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier

C:\Users\DELL XPS\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
C:\Users\DELL XPS\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [2]:
gold = yf.download("GC=F")
silver = yf.download("SI=F")
copper = yf.download("HG=F")
platinum = yf.download("PL=F")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [11]:
def compare(symb):
    a = yf.download(symb)
    c = a.astype(float)
    b = pd.DataFrame(columns = ['Values'])
    for index, row in c.iterrows():
        b['Values'] = np.where((a['Open'] <= a['Close']), "1","0")
        
    b.dropna(inplace=True)
    return b

In [12]:
gg = compare("GC=F")
ss = compare("SI=F")
cc = compare("HG=F")
pp = compare("PL=F")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [13]:
def close(df,name):
    aa = df[['Close']]
    aa.columns = [[name]]
    tmp = pd.DataFrame(aa).dropna()
    return tmp

def openp(df,name):
    bb = df[['Open']]
    bb.columns = [[name]]
    ext = pd.DataFrame(bb).dropna()
    return ext

In [14]:
g_c = close(gold,'gold_close')
s_c = close(silver, 'silver_close')
c_c = close(copper, 'copper_close')
p_c = close(platinum, 'platinum_close')
g_o = openp(gold, 'g_open')
s_o = openp(silver, 's_open')
c_o = openp(copper, 'c_open')
p_o = openp(platinum, 'p_open')

In [15]:
son_gold = g_o.join([g_c,s_c,c_c,p_c]).dropna()
son_sil = s_o.join([g_c,s_c,c_c,p_c]).dropna()
son_cop = c_o.join([g_c,s_c,c_c,p_c]).dropna()
son_pla = p_o.join([g_c,s_c,c_c,p_c]).dropna()

#### Gold prices against opening prices of gold, silver, copper, and platinum :

In [16]:
X = son_pla.iloc[:,1:]
X

,gold_close,silver_close,copper_close,platinum_close
Date,,,,
2000-08-30 00:00:00-04:00,273.899994,4.930000,0.8850,591.400024
2000-08-31 00:00:00-04:00,278.299988,5.003000,0.8850,586.700012
2000-09-01 00:00:00-04:00,277.000000,5.004000,0.8890,595.299988
2000-09-05 00:00:00-04:00,275.799988,4.998000,0.9060,601.299988
2000-09-06 00:00:00-04:00,274.200012,4.983000,0.9015,611.099976
...,...,...,...,...
2022-11-01 00:00:00-04:00,1645.000000,19.673000,3.5095,959.799988
2022-11-02 00:00:00-04:00,1645.699951,19.600000,3.5055,960.200012
2022-11-03 00:00:00-04:00,1627.300049,19.436001,3.4565,933.400024


In [26]:
y = gg

,Values
0,1
1,1
2,1
3,1
4,1
...,...
5563,1
5564,0
5565,0
5566,1


In [21]:
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.75)

model = LogisticRegression(max_iter=1500)
model.fit(X_train,y_train)
print(model.score(X_test,y_test))
y_predict = model.predict(X_test)
confusion_matrix(y_test,y_predict)

ValueError: Found input variables with inconsistent numbers of samples: [4865, 5568]

In [ ]:
def bootstrap(X,y,model):
    res = []
    for i in range(100):
        X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.75)
        model.fit(X_train,y_train)
        res.append(model.score(X_test,y_test))
    tmp = sorted(res)[3:97]
    return (min(tmp),max(tmp))

In [27]:
model = DecisionTreeRegressor()
bootstrap(X,y,model)

ValueError: Found input variables with inconsistent numbers of samples: [4865, 5568]

In [28]:
model = logit('g_open ~ gold_close + silver_close + copper_close + platinum_close', data=son_gold).fit()
model.summary()

ValueError: endog must be in the unit interval.

#### I was going to test these models above, but unfortunately I couldn't fix ValueError.